In [1]:
## Speaker Modelling
# Rewritten from a MATALB script I wrote as part of an acoustics class

In [11]:
# Imports
import numpy as np
import math
from matplotlib import pyplot as plt

In [6]:
rho = 1.2
co = 344
Zo = rho*co

In [8]:
diam = 0.107                          # diameter of the driver
Ad = np.pi * diam * diam / 4          # cross sectional area of the driver
a = diam/2

m = 0.009                             # moving mass is 9 grams
Cm = 740e-6                           # compliance in meters / Newton
omres_driver = math.sqrt(1/(m*Cm))         # mech resonance of driver
fres_driver = 1/(2*np.pi) * omres_driver 
Qfact = 2.21                          # quality factor of resonance for driver
Rm = omres_driver * m/Qfact           # obtaining Rm from Qfact 

In [9]:
Vs = 2.83                             # RMS voltage

BL = 5.9                              # BL constant for the driver system
Re = 6                                # resistance in ohms
L = 0.27e-3                           # inductance in Henry
Vencl = 0.0638                        # enclosed volume in m^2

Ca = Vencl / (Zo * co)                # acoustic compliance
Zarp = 885
Zard = 885
Rap = 0
ma = 35.21

In [16]:
def Zpar2(Z1,Z2): #computes parallel impedance

    return ((Z1*Z2)/(Z1+Z2))

In [21]:
# Set up frequency space
f = np.logspace(0,3,500)
f_len = len(f)
Qd = np.zeros((f_len, 1))
Qp = np.zeros((f_len, 1))
Qtot = np.zeros((f_len, 1))
P = np.zeros((f_len, 1))
i = np.array([1j]) # The imaginary number

In [26]:
# Model speaker response at each frequency
for n in range(f_len):
    
    om = 2*np.pi*f[n]
    i_om = np.multiply(i, om)
    
    F = BL*Vs / (Re + i_om*L)

    Pd = F / Ad;
    
    Zmd = Rm + i_om*m + 1/(i_om*Cm) + Zpar2((BL*BL/L)/i_om , (BL*BL/Re))
    
#     Z1 = Zard + Zmd / (Ad^2)
#     Z2 = Zarp + Rap + i*om*ma # 1e9 used for closed cabinet (ideally infinite)
#     Z3 = 1 / (i*om*Ca)
    
#     print(Z1)
    
#     Z_MAT = [(Z1 + Z3) Z1 ; Z1 (Z1 + Z2)]
#     P_MAT = [Pd; Pd]
    
#     Q_MAT = Z_MAT \ P_MAT;
    
#     Qd(n) = Q_MAT(1) + Q_MAT(2)
#     Qp(n) = - Q_MAT(2)
    
#     Qtot(n) = Qd(n) + Qp(n)
    
#     P(n) = i*rho*om*Qtot(n) / (4*np.pi*1) # at 1 meter radius



[7.37968845-215.01933914j]
[7.37968844-212.06177159j]
[7.37968842-209.14484297j]
[7.37968841-206.26799427j]
[7.37968839-203.43067418j]
[7.37968838-200.63233897j]
[7.37968836-197.87245237j]
[7.37968835-195.15048548j]
[7.37968833-192.46591668j]
[7.37968832-189.8182315j]
[7.3796883-187.20692255j]
[7.37968828-184.63148941j]
[7.37968827-182.09143852j]
[7.37968825-179.58628311j]
[7.37968823-177.11554311j]
[7.37968821-174.67874503j]
[7.37968819-172.27542189j]
[7.37968817-169.90511312j]
[7.37968815-167.56736449j]
[7.37968813-165.26172799j]
[7.37968811-162.98776178j]
[7.37968808-160.74503008j]
[7.37968806-158.5331031j]
[7.37968804-156.35155696j]
[7.37968801-154.19997358j]
[7.37968799-152.07794065j]
[7.37968796-149.98505151j]
[7.37968793-147.92090507j]
[7.37968791-145.88510578j]
[7.37968788-143.8772635j]
[7.37968785-141.89699344j]
[7.37968782-139.94391612j]
[7.37968779-138.01765726j]
[7.37968776-136.11784771j]
[7.37968772-134.2441234j]
[7.37968769-132.39612525j]
[7.37968766-130.57349912j]
[7.379

[7.0304049+49.00019879j]
[7.02120525+49.69224809j]
[7.01177965+50.39394444j]
[7.00212341+51.10542689j]
[6.9922318+51.82683654j]
[6.98210005+52.55831655j]
[6.97172332+53.30001214j]
[6.96109674+54.05207068j]
[6.9502154+54.81464169j]


In [ ]:
figure (1)
loglog(f,abs(Qtot),'b');
grid on;
hold on;
loglog(f,abs(Qp),'r');
loglog(f,abs(Qd),'m');
xlim([1 1000]);
xlabel('Frequency (Hz)');
ylabel('Volume Velocity [m^3 / sec]');
legend('Qtot', 'Qp', 'Qd');
title('Volume Velocities vs Frequency');
pedit;

figure(2);
%SPL = 20*log10(abs(P)/20e-6); % Used for calculating bass reflex SPL 
load('bass_response_spl.mat'); % load bass reflex SPL for plotting
SPL_closed = 20*log10(abs(P)/20e-6);
semilogx(f,SPL,'r'); hold on;
semilogx(f,SPL_closed,'k');
grid on;
xlim([20 1000]);
ylim([50 100]);
xlabel('Frequency (Hz)');
ylabel('SPL dB');
legend('Bass Reflex', 'Closed Cabinet');
title('SPL vs Frequency');
pedit;